In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('/Users/evgeniya/Desktop/Рекомендательные системы/2/data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
my_result = data_test.groupby('user_id')['item_id'].unique().reset_index()
my_result.columns=['user_id', 'actual']
my_result['actual'] = my_result['actual'].apply(lambda x: list(x))

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
Зачем он нужен?  
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
Можно ли удалить этот товар?   
Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

На вебинаре мы брали только 5000 самых популярных товаров, остальные кодировали как 999999. Это было сделано, чтобы не появились пользователи без покупок. В противном случае мы бы их потеряли.

In [5]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

/Users/evgeniya/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [6]:
data_train.shape

(2278490, 12)

In [7]:
# Создадим датафрейм без товаров 999999

data_whithout_9 = data_train.drop(data_train[data_train.item_id == 999999].index)
data_whithout_9.shape

(1448716, 12)

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [8]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights.iloc[:,0])
    probability = np.array(items_weights.iloc[:,1])
    recs = np.random.choice(items, size=n, replace=True, p=probability)
    
    return recs.tolist()

Сделайте предсказания

In [9]:
sales_log = data_train.groupby('item_id')['sales_value'].sum().reset_index()    

sales_log['sales_value'] = np.log(sales_log['sales_value'], out=np.zeros_like(sales_log['sales_value']), where=(sales_log['sales_value'] !=0))

sales_log.loc[sales_log['sales_value'] < 0, 'sales_value'] = 0

sales_log['sales_value'] = sales_log['sales_value']/(sales_log['sales_value'].sum())

my_result['weighted_random_recs_1'] = my_result['user_id'].apply(lambda x: weighted_random_recommendation(sales_log, n=5))

In [10]:
sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()
sales['sales_value'] = sales['sales_value']/(sales['sales_value'].sum())

my_result['weighted_random_recs_2'] = my_result['user_id'].apply(lambda x: weighted_random_recommendation(sales, n=5))

In [11]:
sales_num = data_train.groupby('item_id')['quantity'].sum().reset_index()
sales_num['quantity'] = sales_num['quantity']/(sales_num['quantity'].sum())

my_result['weighted_random_recs_3'] = my_result['user_id'].apply(lambda x: weighted_random_recommendation(sales_num, n=5))

In [12]:
my_result.head(5)

,user_id,actual,weighted_random_recs_1,weighted_random_recs_2,weighted_random_recs_3
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1122928, 1127141, 912949, 1032512, 990762]","[6534178, 999999, 903325, 999999, 918598]","[6534178, 6534178, 6534178, 6534178, 6534178]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[891505, 904360, 929668, 5580793, 828525]","[999999, 999999, 981760, 1137344, 859075]","[6534166, 6534178, 6534178, 6534178, 6534178]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[950824, 5587502, 827919, 5568729, 1048563]","[8019977, 1020404, 999999, 977497, 999999]","[6534178, 6534178, 6534178, 6534178, 6534178]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[827271, 1075214, 849505, 912949, 976998]","[840386, 999999, 999999, 981288, 999999]","[6534178, 6534166, 6534178, 6534178, 6534178]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[936924, 898068, 1105488, 897088, 1071939]","[999999, 1035676, 999999, 999999, 1029252]","[6534178, 6534178, 6534178, 6534178, 6533889]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [13]:
result = pd.read_csv('/Users/evgeniya/Desktop/Рекомендательные системы/2/preds.csv') # закгрузка predict с семианара

result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13039087, 1190490, 6552030, 9884683, 1617215]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[965430, 664229, 829690, 4519071, 1015781]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [14]:
type(result['actual'][0])

str

In [15]:
# файл с предсказаниями сохранил предсказания как строки
# нужно перевести обратно в список
for a in result.columns[1:]:
    result[a]=  result[a].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

In [16]:
result['weighted_random_recs_1'] = my_result['weighted_random_recs_1']
result['weighted_random_recs_2'] = my_result['weighted_random_recs_2']
result['weighted_random_recs_3'] = my_result['weighted_random_recs_3']

result.head(3)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recs_1,weighted_random_recs_2,weighted_random_recs_3
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13039087, 1190490, 6552030, 9884683, 1617215]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1122928, 1127141, 912949, 1032512, 990762]","[6534178, 999999, 903325, 999999, 918598]","[6534178, 6534178, 6534178, 6534178, 6534178]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[965430, 664229, 829690, 4519071, 1015781]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[891505, 904360, 929668, 5580793, 828525]","[999999, 999999, 981760, 1137344, 859075]","[6534166, 6534178, 6534178, 6534178, 6534178]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[2664434, 74980, 864943, 9337133, 77162]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]","[950824, 5587502, 827919, 5568729, 1048563]","[8019977, 1020404, 999999, 977497, 999999]","[6534178, 6534178, 6534178, 6534178, 6534178]"


In [17]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list     
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [18]:
alg = list(result.columns)[2:] 
for i in alg:
    precision = result.apply(lambda x: precision_at_k(x[i], x['actual'],  5), axis=1).mean()
    print(i, precision)

random_recommendation 0.0008814887365328109
popular_recommendation 0.15523996082272082
itemitem 0.13692458374142857
cosine 0.13290891283055686
tfidf 0.1389813907933383
own_purchases 0.17969311132876015
weighted_random_recs_1 0.006660137120470119
weighted_random_recs_2 0.023016650342801218
weighted_random_recs_3 0.04113614103819817


own_purchases показал наилучшее качество

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


На вебинаре мы давали рекомендации, закодировав товары не из топ-5000 как 99999.

Сделаем рекомендации и посчитаем precision@5, на основе датасета, где удалены все строки с номером товара 99999.

In [19]:
user_item_matrix = pd.pivot_table(data_whithout_9, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 
user_item_matrix = user_item_matrix.astype(float) 

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [20]:
top_result = data_whithout_9.groupby('user_id')['item_id'].unique().reset_index()
top_result.columns=['user_id', 'actual']
top_result['actual'] = top_result['actual'].apply(lambda x: list(x))

In [21]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [22]:
items = data_whithout_9.item_id.unique()

top_result['random_recommendation'] = top_result['user_id'].apply(lambda x: random_recommendation(items, n=5))

In [23]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [24]:
popular_recs = popularity_recommendation(data_whithout_9, n=5)

top_result['popular_recommendation'] = top_result['user_id'].apply(lambda x: popular_recs)

In [25]:
model = ItemItemRecommender(K=5, num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  
                        user_items=csr_matrix(user_item_matrix).tocsr(),
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

In [26]:
top_result['itemitem'] = top_result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

In [27]:
model = CosineRecommender(K=5, num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

In [28]:
top_result['cosine'] = top_result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

In [29]:
model = TFIDFRecommender(K=5, num_threads=4) 

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

In [30]:
top_result['tfidf'] = top_result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,  
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

In [31]:
model = ItemItemRecommender(K=3, num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  
                        user_items=csr_matrix(user_item_matrix).tocsr(),
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

top_result['itemitem_3'] = top_result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

In [32]:
model = ItemItemRecommender(K=7, num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  
                        user_items=csr_matrix(user_item_matrix).tocsr(),
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

top_result['itemitem_7'] = top_result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

In [33]:
top_result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,itemitem_3,itemitem_7
0,1,"[840361, 845307, 852014, 856942, 912676, 94094...","[12731808, 1098310, 1056964, 851865, 864532]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 995242, 840361, 1127831]"
1,2,"[854852, 930118, 1077555, 1098066, 5568489, 55...","[9445549, 1126899, 1092552, 965842, 9419525]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 1127831, 995242]","[1082185, 981760, 1098066, 1127831, 826249]","[1082185, 981760, 1127831, 1098066, 826249]","[1082185, 981760, 1127831, 1098066, 826249]","[1082185, 981760, 1098066, 995242, 840361]"
2,3,"[866211, 878996, 882830, 904360, 921345, 93779...","[6391249, 1020156, 5591627, 966891, 995965]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 1127831]","[1082185, 981760, 1098066, 826249, 995242]"
3,4,"[857849, 883932, 891423, 897125, 901032, 90497...","[6552253, 1029684, 1098708, 1112766, 850281]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 883404]","[981760, 1082185, 1098066, 826249, 883404]","[1082185, 981760, 1029743, 840361, 883404]","[1082185, 981760, 1098066, 826249, 995242]"
4,5,"[889509, 937626, 941797, 1004596, 1114597, 112...","[931672, 1051781, 8293732, 1016539, 968992]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 961554]","[1082185, 981760, 961554, 1098066, 826249]","[1082185, 981760, 1029743, 961554, 916122]","[1082185, 981760, 1098066, 995242, 826249]"


In [34]:
alg_2 = list(top_result.columns)[2:] 
for i in alg_2:
    precision = top_result.apply(lambda x: precision_at_k(x[i], x['actual'],  5), axis=1).mean()
    print(i, precision)

random_recommendation 0.05390185036202773
popular_recommendation 0.3950925181013693
itemitem 0.6536604987932366
cosine 0.7025744167337019
tfidf 0.7489139179404554
itemitem_3 0.789474389916849
itemitem_7 0.6510056315365994


Precision@5 значительно увеличился по сравнению с первоначальным вариантом.

Так же Precision@5 существенно увеличился для ItemItemRecommender при уменьшении количества соседей  𝐾 до 3.

### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.